### This Notebook will create the Decision Tree algorithm from scratch. The dataset used for this will be the sklearn diabetes which is a regression problem.

In [101]:
from sklearn import datasets
import numpy as np
from sklearn.model_selection import train_test_split

In [102]:
ds = datasets.load_diabetes()

In [103]:
X = np.array(ds.data)
y = np.array(ds.target)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
X_train

array([[-0.00551455, -0.04464164,  0.04229559, ..., -0.03949338,
         0.05227699,  0.02791705],
       [ 0.06350368, -0.04464164, -0.05039625, ...,  0.02360753,
         0.05803805,  0.04034337],
       [ 0.0090156 , -0.04464164,  0.05522933, ...,  0.02323852,
         0.05568623,  0.10661708],
       ...,
       [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
        -0.01090325, -0.0010777 ],
       [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
        -0.03845972, -0.03835666],
       [-0.09269548, -0.04464164,  0.02828403, ..., -0.03949338,
        -0.00514219, -0.0010777 ]])

Now that we have the data, we need to make the decision tree. The decision tree separates the data into different leafs. It does this by separating the data using different criteria. For example, on the first iteration we might separate the rows that have the value of index[0] less than 0 to the left leaf and the ones that have more or equal than 0 to the right leaf. But how do we decide where to split the data? 

 Since we don't know the best split criteria, we will have to find it. The most basic way is by iterating through each column and for each column we test every possible value to split and see which one is the best. However, if a column A can have all possible positive values, there would be infinite values for which to try to split the data. So, to circunvent this, for each column, we try to separate the data using only the values we have in our training data. But now, how do we determine what is the best split. There are different metrics that we can use. For example, in the sklearn DecisionTreeRegressor the defalut metric is the mean squared error (MSE). The best split is the one that has the lower MSE. 

So, let's build 5 functions:
 1) Calculates the MSE of a list of values.
 2) For a given number of lists (in this case it's always 2) it uses the previous function to calculate the MSE in each list and sums the values of the MSEs.
 3) For a given list and a value, it splits the list into 2 lists: one where the values are less than the provided value and the other where the values are equal or bigger.
 4) For a given list it calculates the split that results in the least MSE
 5) For a matrix X and labels y, it loops through all the columns and executes function 4), calculating the best column and value to split on, returning a dictionary with the index of the column, the value and the MSE

In [104]:
def calculate_MSE(lst:list)->float:
    if len(lst) == 0:
        MSE = float('inf')
    else:
        total = 0
        mean = np.mean(lst)
        total = sum([(value-mean)**2 for value in lst])
        MSE = total/len(lst)
    return MSE

In [105]:
def calculate_lists_MSEs(list_of_lists:list[list[float]])->float:
    MSE_sum = 0
    len_all = sum([len(ls) for ls in list_of_lists])
    MSE_sum = sum([len(number_list)/len_all*calculate_MSE(number_list) for number_list in list_of_lists])
    return MSE_sum

In [106]:
def calculate_split_indexes(split_value:float, list_column_values:list[float])->list[list[int], list[int]]:
    indexes_smaller = []
    indexes_eq_bigger = []
    for index, value in enumerate(list_column_values):
        if value<split_value:
            indexes_smaller.append(index)
        else:
            indexes_eq_bigger.append(index)
    return [indexes_smaller, indexes_eq_bigger]


In [107]:
def calculate_best_column_MSE(float_list:np.ndarray[float], labels:np.ndarray[float], min_MSE:float)->tuple[float, float]:
    best_split = None
    for value in float_list:
        list_of_lists_of_indexes= calculate_split_indexes(value, float_list)
        labels_lists = [labels[indexes] for indexes in list_of_lists_of_indexes]
        value_MSE = calculate_lists_MSEs(labels_lists)
        if value_MSE< min_MSE:
            min_MSE = value_MSE
            best_split = value
    return (best_split, min_MSE)


In [108]:
def calculate_matrix_best_split(matrix: np.ndarray[float], labels: np.ndarray[float])->dict[str, float]:
    result_dictionary = {
        "column_index":-1,
        "split":-1,
        "MSE":calculate_MSE(labels)
    }
    for column_index in range(matrix.shape[1]):
        column_values = matrix[:, column_index] #select all values from the column of index column_index
        split_value, MSE = calculate_best_column_MSE(column_values, labels, result_dictionary['MSE'])
        if MSE<result_dictionary['MSE']:
            result_dictionary['column_index'] = column_index
            result_dictionary['split'] = split_value
            result_dictionary['MSE'] = MSE
    return result_dictionary

### Now let's try to determine the best split for our data!

In [109]:
print(f"Train data MSE: {round(np.var(y_train), 2)}.")
split_dct = calculate_matrix_best_split(X_train, y_train) 
print(f"Splited dadta MSE: {round(split_dct['MSE'], 2)}")
print(f"Column used to split the data: {split_dct['column_index']}.")
print(f"Value to split the data on: {split_dct['split']}.")

Train data MSE: 6044.62.


Splited dadta MSE: 4288.93
Column used to split the data: 2.
Value to split the data on: 0.005649978676881689.


Now we just need to keep doing this iteratively. Now the question is: Untill when do we keep doing this? We can do this until we can't do anymore split, (when every leaf node has only one label, multiple labels all with the same value or when the values are different but the feature rows values are all the same), when we reach a maximum number of iterations that we define, or we reach the maximum number of leaves that we define. In this notebook we will build one tree that does the split until we can't do it anymore. In the next ones we will go through different parameters and add them to our Decision tree model. 

First we have to alter the function calculate_matrix_best_split and calculate_best_column_MSE to also return the splits. 

In [110]:
def calculate_best_column_MSE(float_list:np.ndarray[float], labels:np.ndarray[float], min_MSE:float)->tuple[float, float]:
    best_split = None
    indexes_1 = np.empty((0))
    indexes_2 = np.empty((0))
    for value in float_list:
        list_of_lists_of_indexes= calculate_split_indexes(value, float_list)
        labels_lists = [labels[indexes] for indexes in list_of_lists_of_indexes]
        value_MSE = calculate_lists_MSEs(labels_lists)
        if value_MSE< min_MSE:
            min_MSE = value_MSE
            best_split = value
            indexes_1 = list_of_lists_of_indexes[0]
            indexes_2 = list_of_lists_of_indexes[1]
    return (best_split, min_MSE,indexes_1, indexes_2)


In [111]:
def calculate_matrix_best_split(matrix: np.ndarray[float], labels: np.ndarray[float])->dict[str, float]:
    result_dictionary = {
        "split":None,
        "MSE":calculate_MSE(labels),
    }
    for column_index in range(matrix.shape[1]):
        column_values = matrix[:, column_index] #select all values from the column of index column_index
        split_value, MSE, indexes_1, indexes_2 = calculate_best_column_MSE(column_values, labels, result_dictionary['MSE'])
        if MSE<result_dictionary['MSE']:
            result_dictionary['column_index'] = column_index
            result_dictionary['split'] = split_value
            result_dictionary['MSE'] = MSE
            result_dictionary["X1"] = matrix[indexes_1]
            result_dictionary["X2"] = matrix[indexes_2]
            result_dictionary["y1"] = labels[indexes_1]
            result_dictionary["y2"] = labels[indexes_2]
    return result_dictionary

In [112]:
calculate_matrix_best_split(X_train, y_train)

{'split': 0.005649978676881689,
 'MSE': 4288.930696708918,
 'column_index': 2,
 'X1': array([[ 0.06350368, -0.04464164, -0.05039625, ...,  0.02360753,
          0.05803805,  0.04034337],
        [-0.08906294, -0.04464164, -0.01159501, ...,  0.03430886,
          0.02268774, -0.00936191],
        [-0.0382074 , -0.04464164, -0.0105172 , ..., -0.00259226,
         -0.01811369, -0.01764613],
        ...,
        [-0.09632802, -0.04464164, -0.07626374, ..., -0.03949338,
         -0.05947118, -0.08391984],
        [ 0.03081083, -0.04464164, -0.02021751, ..., -0.03949338,
         -0.01090325, -0.0010777 ],
        [-0.01277963, -0.04464164, -0.02345095, ..., -0.00259226,
         -0.03845972, -0.03835666]]),
 'X2': array([[-0.00551455, -0.04464164,  0.04229559, ..., -0.03949338,
          0.05227699,  0.02791705],
        [ 0.0090156 , -0.04464164,  0.05522933, ...,  0.02323852,
          0.05568623,  0.10661708],
        [ 0.01628068,  0.05068012,  0.01427248, ...,  0.03430886,
          0.

Now to finish this, we will recursively calculate the best split of the matrix. We stop this until the branch where we are in only has one value in the leaf or when there is no split that will improve the MSE of the values inside the leaf.

In [113]:
import numpy as np

def split_data_best_split(X: np.ndarray, y: np.ndarray)->dict:
    tree = {}
    best_split_dct = calculate_matrix_best_split(X, y)
    if best_split_dct["split"] is None:
        tree["value"] = y
    else:
        branch_1 = (best_split_dct["X1"], best_split_dct["y1"])
        branch_2 = (best_split_dct["X2"], best_split_dct["y2"])

        tree['column_index'] = best_split_dct['column_index']
        tree['split'] = best_split_dct['split']
        if len(branch_1[0]) > 1:
            tree['left'] = split_data_best_split(branch_1[0], branch_1[1])
        else:
            tree['left'] = {'value': branch_1[1]}  # Assuming leaf node value
        if len(branch_2[0]) > 1:
            tree['right'] = split_data_best_split(branch_2[0], branch_2[1])
        else:
            tree['right'] = {'value': branch_2[1]}  # Assuming leaf node value

    return tree

In [114]:
res = split_data_best_split(X_train, y_train)

Nice, we were now able to separate the different samples and reduced the MSE. If we now want to calculate the label y for an observation we just have to follow the tree and reach the leaf node. When we reach it, we calculate the average of the values of the leaf node and that is the value we want! Let's try

In [115]:
def predict(x:np.ndarray, tree:dict)->float:
    if "value" in tree.keys():
        return np.mean(tree['value'])
    else:
        if x[tree['column_index']]<tree["split"]:
            return predict(x, tree['left'])
        else:
            return predict(x, tree['right'])

In [116]:
predict(X_train[0], res)

166.0

In [117]:
y_train[0]

166.0

It works! The problem with this tree right now it's that it has overfitted the data that we gave it. The MSE of the training data is perfect but the MSE of the test data will be awful:

In [118]:
y_pred = [predict(x, res) for x in X_train]
error_train = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(y_pred, y_train)) / len(X_train)
y_pred = [predict(x, res) for x in X_test]
error_test = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(y_pred, y_test)) / len(X_test)
error_train

print(f"Train data MSE: {error_train}")
print(f"Test data MSE: {round(error_test, 2)}")

Train data MSE: 0.0
Test data MSE: 6103.76


But hey, we successfully buit a Decision tree from scratch. Let's now compare with sklearn's decision tree with the same parameters as ours tree and see if the results are similar.

In [119]:
from sklearn.tree import DecisionTreeRegressor
tr = DecisionTreeRegressor(random_state=42)
tr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [120]:
preds = tr.predict(X_train)
error_train = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(preds, y_train)) / len(X_train)
error_train
preds = tr.predict(X_test)
error_test = sum((y_train_pred -y_true)**2 for y_train_pred, y_true in zip(preds, y_test)) / len(X_test)
error_test
print(f"Train data MSE: {error_train}")
print(f"Test data MSE: {round(error_test, 2)}")

Train data MSE: 0.0
Test data MSE: 5941.7


Pretty much the same result (Although it's much faster)! In the next notebook we change our functions and implement some different parameters to decrease ou overfitting.